# **2D Ising Model**

In [1]:
%reload_ext Cython
%matplotlib widget

In [2]:
import numpy as np
from PIL import Image
from ipywidgets import interact, FloatSlider, Button, Output, IntSlider, VBox, HBox
import matplotlib.pyplot as plt
from time import sleep

def random_spin_field(N, M):
    return np.random.choice([-1, 1], size=(N, M))

def display_spin_field(field):
    return Image.fromarray(np.uint8((field + 1) * 0.5 * 255))  # 0 ... 255

def display_ising_sequence(images):
    def _show(frame=(0, len(images) - 1)):
        return display_spin_field(images[frame])
    return interact(_show)

Jvalue = FloatSlider(value = -1.0, min = -1.0, max = 1.0, description = 'Exchange interaction J:',
                    style={'description_width': 'initial'})
Brun = Button(description='Run simulation')
Brun.style.button_color = 'lightgreen'

Frame = IntSlider(value=0, min=0, max=100, description="Frame", layout={'width':'500px'}, disabled=True)
Num = IntSlider(value=50, min=50, max=500, step=50, description="N")

In [3]:
%%cython

cimport cython

import numpy as np
cimport numpy as np

from libc.math cimport exp
from libc.stdlib cimport rand
cdef extern from "limits.h":
    int RAND_MAX


@cython.boundscheck(False)
@cython.wraparound(False)
def cy_ising_step(np.int64_t[:, :] field, float J, float beta=0.4):
    cdef int N = field.shape[0]
    cdef int M = field.shape[1]
    cdef int n_offset, m_offset, n, m
    for n_offset in range(2):
        for m_offset in range(2):
            for n in range(n_offset, N, 2):
                for m in range(m_offset, M, 2):
                    _cy_ising_update(field, n, m, beta, J)
    return np.array(field)

@cython.boundscheck(False)
@cython.wraparound(False)
cdef _cy_ising_update(np.int64_t[:, :] field, int n, int m, float beta, float J):
    cdef int total = 0
    cdef int N = field.shape[0]
    cdef int M = field.shape[1]
    cdef int i, j
    for i in range(n-1, n+2):
        for j in range(m-1, m+2):
            if i == n and j == m:
                continue
            total += field[i % N, j % M]
    cdef float dE = -2.0* J * field[n, m] * total
    if dE <= 0:
        field[n, m] *= -1
    elif exp(-dE * beta) * RAND_MAX > rand():
        field[n, m] *= -1

In [4]:
def run_simulation(b):
    Brun.style.button_color = 'red'
    global images
    images = [random_spin_field(Num.value, Num.value)]
    for i in range(100):
        images.append(cy_ising_step(images[-1].copy(), beta=0.4, J=Jvalue.value))
    fig.set_data(images[0])
    Frame.value = 0
    Frame.disabled = False
    Brun.style.button_color = 'lightgreen'

def on_frame_change(b):
    fig.set_data(images[Frame.value])
        

Brun.on_click(run_simulation);

Frame.observe(on_frame_change, names='value')

images = [random_spin_field(Num.value, Num.value)]

img, ax = plt.subplots()
img.canvas.header_visible = False
fig = ax.imshow(images[0])



display(HBox([Jvalue, Num]), Brun, Frame)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Run simulation', style=ButtonStyle(button_color='lightgreen'))

IntSlider(value=0, description='Frame', disabled=True, layout=Layout(width='600px'), max=200)